<a href="https://colab.research.google.com/github/NikkiIndra/CORE-City-Operation-Response-Ecosystem/blob/main/Korensi_Otomatis_File_arduino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Install modul yang dibutuhkan
!pip install rarfile > /dev/null
!apt-get install unrar -y > /dev/null

import zipfile, os, glob, requests, re, csv, shutil
from google.colab import files
import rarfile


In [11]:
import os
os.listdir()


['.config', 'hasil_penilaian', 'extracted', 'sample_data']

In [ ]:
from google.colab import files
import zipfile, os

# Upload file ZIP dulu
uploaded = files.upload()  # ← Pilih file seperti 14522634_Novan_TI2.zip

# Ganti ke nama file yang kamu upload
ZIP_FILE = "14522634_Novan_TI2.zip"
EXTRACT_DIR = "extracted"

# Ekstrak
os.makedirs(EXTRACT_DIR, exist_ok=True)
with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)

print(f"✅ Berhasil ekstrak {ZIP_FILE} ke folder {EXTRACT_DIR}")


In [31]:
!apt-get install unrar -y > /dev/null 2>&1

import zipfile, os, glob, requests, re, csv, shutil
from collections import defaultdict
from google.colab import files
import rarfile

# === Konfigurasi ===
API_KEY = "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"
API_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL = "mistralai/mistral-7b-instruct"

EXTRACT_DIR = "extracted"
HASIL_DIR = "hasil_penilaian"
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(HASIL_DIR, exist_ok=True)

# === Upload file zip/rar ===
uploaded = files.upload()

# === Fungsi Bantu ===
def baca_template(soal_ke):
    return open(f"prompt_template_soal{soal_ke}.txt", "r", encoding="utf-8").read()

def deteksi_soal(path):
    path = path.lower().replace(" ", "_")
    parts = path.split(os.sep)
    for part in reversed(parts):
        if any(k in part for k in ["soal_1", "no_1", "soal1", "no1", "praktikum_no1", "praktikumno1", "1"]): return 1
        if any(k in part for k in ["soal_2", "no_2", "soal2", "no2", "praktikum_no2", "praktikumno2", "2"]): return 2
        if any(k in part for k in ["soal_3", "no_3", "soal3", "no3", "praktikum_no3", "praktikumno3", "3"]): return 3
    return None

def ambil_nilai(hasil):
    skor = 0
    komentar = ""
    # Koreksi jika komentar ada tapi skor 0 (kemungkinan parsing gagal)
    if skor == 0 and komentar and not "gagal" in komentar.lower():
        skor = 50  # estimasi default jika parsing gagal tapi komentar ada

    skor_match = re.search(r"(?:skor|nilai)\s*[:\-]?\s*(\d+)", hasil, re.IGNORECASE)
    if skor_match:
        skor = int(skor_match.group(1))

    komentar_match = re.search(r"(?:alasan penilaian|komentar|penjelasan)\s*[:\-]?\s*(.*)", hasil, re.IGNORECASE | re.DOTALL)
    if komentar_match:
        komentar = komentar_match.group(1).strip().split("\n")[0]

    return skor, komentar


def infer_ai(prompt):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }
    data = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "Kamu adalah dosen penilai tugas Arduino."},
            {"role": "user", "content": prompt}
        ]
    }
    r = requests.post(API_URL, headers=headers, json=data)
    try:
        return r.json()["choices"][0]["message"]["content"]
    except Exception as e:
        print("❌ Gagal inferensi:", r.text)
        return None

# === Penilaian ===
rekap = []

for file in uploaded.keys():
    nama_mahasiswa = os.path.splitext(file)[0]
    tujuan_ekstrak = os.path.join(EXTRACT_DIR, nama_mahasiswa)
    os.makedirs(tujuan_ekstrak, exist_ok=True)

    try:
        if file.endswith(".zip"):
            with zipfile.ZipFile(file, 'r') as zip_ref:
                zip_ref.extractall(tujuan_ekstrak)
        elif file.endswith(".rar"):
            with rarfile.RarFile(file, 'r') as rar_ref:
                rar_ref.extractall(tujuan_ekstrak)
        else:
            print(f"❌ Format tidak didukung: {file}")
            continue
    except Exception as e:
        print(f"❌ Gagal ekstrak {file}: {e}")
        continue

    # Struktur {1: skor, 2: skor, 3: skor}
    skor_soal = {}
    komentar_soal = {}

    for ino_path in glob.glob(f"{tujuan_ekstrak}/**/*.ino", recursive=True):
        print(f"🔍 Menilai: {ino_path}")
        if os.path.isdir(ino_path): continue

        soal_ke = deteksi_soal(ino_path)
        if not soal_ke:
            print("⚠ Soal tidak dikenali")
            continue

        kode = open(ino_path, "r", encoding="utf-8", errors="ignore").read()
        template = baca_template(soal_ke)
        prompt = template.replace("{JAWABAN}", kode)

        hasil = infer_ai(prompt)
        if not hasil:
            skor_soal[soal_ke] = 0
            komentar_soal[soal_ke] = "Gagal inferensi"
            continue

        skor, komentar = ambil_nilai(hasil)
        skor_soal[soal_ke] = skor
        komentar_soal[soal_ke] = komentar

        # Simpan hasil ke folder per mahasiswa
        out_dir = os.path.join(HASIL_DIR, nama_mahasiswa)
        os.makedirs(out_dir, exist_ok=True)
        with open(os.path.join(out_dir, f"soal_{soal_ke}.txt"), "w", encoding="utf-8") as f:
            f.write(hasil)

    # Rekap akhir
    jumlah_soal = len(skor_soal)
    total_skor = sum(skor_soal.values())
    if jumlah_soal == 0:
        total_normalized = 0
    else:
        total_normalized = int((total_skor / 300) * 100) if jumlah_soal == 3 else int((total_skor / (jumlah_soal * 100)) * 100)


    out_dir = os.path.join(HASIL_DIR, nama_mahasiswa)
    os.makedirs(out_dir, exist_ok=True)
    with open(os.path.join(out_dir, f"rekap.txt"), "w", encoding="utf-8") as f:
        f.write(f"Nama Mahasiswa: {nama_mahasiswa}\n")
        for i in range(1, 4):
            f.write(f"\n--- Soal {i} ---\nSkor: {skor_soal.get(i, 0)}\nKomentar: {komentar_soal.get(i, '-')}\n")
        f.write(f"\n\nTotal Nilai: {total_skor} dari maksimal {jumlah_soal * 100} → Skala 100: {total_normalized}\n")

    rekap.append([
        nama_mahasiswa,
        total_normalized,
        komentar_soal.get(1, "-"),
        komentar_soal.get(2, "-"),
        komentar_soal.get(3, "-")
    ])

# Simpan CSV
with open("rekap_nilai.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Nama Mahasiswa", "Total Nilai (maks 100)", "Komentar Soal 1", "Komentar Soal 2", "Komentar Soal 3"])
    writer.writerows(rekap)

print("✅ Semua penilaian selesai. Cek folder hasil_penilaian/ dan file rekap_nilai.csv")


Saving 14523005_Shella_Riyanti_TI-1.rar to 14523005_Shella_Riyanti_TI-1.rar
Saving 14523006_MIMIF_MIFTAHUDDIN_TI-1.zip to 14523006_MIMIF_MIFTAHUDDIN_TI-1.zip
Saving 14523007_Galih_Akbar_TI-1.rar to 14523007_Galih_Akbar_TI-1.rar
Saving 14523008_Ofi_Oktaviani_TI-1.rar to 14523008_Ofi_Oktaviani_TI-1.rar
Saving 14523010_RIFKI_ADITYA_NUGROHO_TI-1.rar to 14523010_RIFKI_ADITYA_NUGROHO_TI-1.rar
Saving 14523011_AGUNG_ARIEF_ROSIDIN_TI-1.rar to 14523011_AGUNG_ARIEF_ROSIDIN_TI-1.rar
Saving 14523012_Fauzia_Khaerani_TI-1.rar to 14523012_Fauzia_Khaerani_TI-1.rar
Saving 14523013_Noval_Rizky_Pratama_TI-1.rar to 14523013_Noval_Rizky_Pratama_TI-1.rar
Saving 14523014_Fatichatus_Saadah_TI-1.zip to 14523014_Fatichatus_Saadah_TI-1.zip
Saving 14523015_MUHAMMAD_ZAINAL_IDRIS_TI-1.rar to 14523015_MUHAMMAD_ZAINAL_IDRIS_TI-1.rar
Saving 14523018_Hikmal_Akbar_TI-1.rar to 14523018_Hikmal_Akbar_TI-1.rar
Saving 14523021_MUHAMMAD_ANNUROFIQ_TI-1.rar to 14523021_MUHAMMAD_ANNUROFIQ_TI-1.rar
Saving 14523025_MUHAMAD_RIFALDI_

untuk hapus semua file yang terupload

In [ ]:
import os
for f in os.listdir():
    if f.endswith(".zip") or f.endswith(".rar"):
        os.remove(f)
        print(f"🗑️ Dihapus: {f}")


HAPUS SEMUA FOLDER EXTRAKTED

In [ ]:
import shutil

shutil.rmtree("extracted", ignore_errors=True)  # Hapus seluruh folder dan isinya
os.makedirs("extracted", exist_ok=True)         # Buat ulang folder kosong
print("✅ Semua isi folder 'extracted/' berhasil dihapus.")


hapus folder Hasil_penilaian

In [ ]:
import shutil
import os

HASIL_DIR = "hasil_penilaian"

# Hapus semua isi folder hasil_penilaian
if os.path.exists(HASIL_DIR):
    shutil.rmtree(HASIL_DIR)
    print(f"✅ Semua isi folder '{HASIL_DIR}' telah dihapus.")
else:
    print(f"⚠️ Folder '{HASIL_DIR}' tidak ditemukan.")

# Buat kembali folder kosong
os.makedirs(HASIL_DIR, exist_ok=True)
print(f"📂 Folder '{HASIL_DIR}' dibuat ulang (kosong).")
